In [ ]:
# Convert the Excel sheets into TSV files

import pandas as pd

#Read excel file into a dataframe
data_xlsx = pd.read_excel('ET Data Postprocessed.xlsx', 'Sheet1', index_col=None)

#Replace all columns having spaces with underscores
data_xlsx.columns = [c.replace(' ', '_') for c in data_xlsx.columns]

#Replace all fields having line breaks with space
df = data_xlsx.replace('\n', ' ',regex=True)

#Write dataframe into csv
df.to_csv('baseStats.csv', sep='\t', encoding='utf-8',  index=False, line_terminator='\r\n')

In [16]:
import pandas as pd
pd.set_option("display.max_rows", None)

df = pd.read_csv('eyetrack_data.tsv', sep='\t', header=0)

# remove the Gender, Timeline, and Recording columns
df = df.drop(columns=['Gender', 'Timeline', 'Recording'])  

# remove rows for the Test Participant
df = df[df['Participant'] != 'Test Participant']

# order by column
df = df.sort_values(by='Participant')

#df.head(3)

KeyError: 'Recording'

In [ ]:
# print total number of rows in data frame
print(len(df))

In [ ]:
# print count
print(df['Recording'].value_counts())

In [4]:
# get summary statistics
#df.agg(["count", "min", "max", "median", "mean", "skew"])